In [ ]:
##################################################
# RFM Analizi İle Müşteri Segmentasyonu
##################################################

In [ ]:
# İş Problemi:
# İngiltere merkezli perakende şirketi müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istemektedir.
# Ortak davranışlar sergileyen müşteri segmentlerini özelinde pazarlama çalışmaları yapmanın gelir artışı sağlayacağını düşünmektedir.
# Segmentlere ayırmak için RFM analizi kullanılacaktır.

In [ ]:
# Online Retail II isimli veri seti İngiltere merkezli bir perakende şirketinin 01/12/2009 - 09/12/2011 tarihleri arasındaki online satış işlemlerini içeriyor.
# Şirketin ürün kataloğunda hediyelik eşyalar yer almaktadır ve çoğu müşterisinin toptancı olduğu bilgisi mevcuttur.

In [ ]:
# Değişkenler

In [ ]:
# InvoiceNo : Fatura Numarası (Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder.)
# StockCode : Ürün Kodu (Her bir ürün için eşsiz)
# Description : Ürün İsmi
# Quantity : Ürün Adedi (Faturalardaki ürünlerden kaçar tane satıldığı)
# InvoiceDate : Fatura Tarihi
# UnitPrice : Fatura Fiyatı (Sterlin)
# CustomerID : Eşsiz Müşteri Numarası
# Country : Ülke İsmi

In [ ]:
##########################################
# Gerekli Kütüphaneler ve Fonksiyonlar
##########################################

In [ ]:
! pip install lifetimes

In [ ]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [ ]:
pd.set_option("display.max_columns", None)  # Tüm sütunları göster
pd.set_option("display.width",500)    # Sütunları aşağıdan inmeden göster
pd.set_option("display.float_format", lambda x: "%.4f" % x)   # Virgülden sonra 4 basamağa kadar göster
from sklearn.preprocessing import MinMaxScaler

In [ ]:
###############################
# Verinin Okunması
################################

In [ ]:
df_ = pd.read_excel("/content/online_retail_II-230817-120704.xlsx",sheet_name="Year 2010-2011")

In [ ]:
df = df_.copy()

In [ ]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,541910.0000,9.5522,-80995.0000,1.0000,3.0000,10.0000,80995.0000,218.0810
InvoiceDate,541910,2011-07-04 13:35:22.342307584,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
Price,541910.0000,4.6111,-11062.0600,1.2500,2.0800,4.1300,38970.0000,96.7598
Customer ID,406830.0000,15287.6842,12346.0000,13953.0000,15152.0000,16791.0000,18287.0000,1713.6031


In [ ]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


In [ ]:
df.isnull().sum()  # ID 'sini bilmediğim bir müşteriyi analiz edemem o yüzden uçuracağız bu kısımları
# Description ksımında eksik verilerde veri setimize oranla çok az o yüzden bunu da uçuracağız

,0
Invoice,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
Price,0
Customer ID,135080
Country,0


In [ ]:
######################################
# Veri Ön İşleme
######################################

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,406830.0000,12.0613,-80995.0000,2.0000,5.0000,12.0000,80995.0000,248.6931
InvoiceDate,406830,2011-07-10 16:31:30.127424512,2010-12-01 08:26:00,2011-04-06 15:02:00,2011-07-31 11:48:00,2011-10-20 13:06:00,2011-12-09 12:50:00,NaN
Price,406830.0000,3.4605,0.0000,1.2500,1.9500,3.7500,38970.0000,69.3151
Customer ID,406830.0000,15287.6842,12346.0000,13953.0000,15152.0000,16791.0000,18287.0000,1713.6031


In [ ]:
# İnvoice değişkeninin başında yer alan C ifadesi iadeleri temsil etmektedir.
# İade olan işlemlerde de Quantity ve Price değişkenlerinde eksi değerler almasına neden olmaktadır.
# Bu yüzden bunları veri setinden çıkarmamız gerekiyor.
df = df[~df["Invoice"].str.contains("C", na=False)]

In [ ]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,397925.0000,13.0218,1.0000,2.0000,6.0000,12.0000,80995.0000,180.4200
InvoiceDate,397925,2011-07-10 23:44:09.817126400,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
Price,397925.0000,3.1162,0.0000,1.2500,1.9500,3.7500,8142.7500,22.0968
Customer ID,397925.0000,15294.3086,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000,1713.1727


In [ ]:
# Eşsiz ürün sayısı
unique_product_count = df["Description"].nunique()
print("Eşsiz ürün sayısı: ", unique_product_count)

Eşsiz ürün sayısı:  3877


In [ ]:
# Hangi üründen kaçar tane var
product_counts = df["Description"].value_counts()
print(product_counts)

Description
WHITE HANGING HEART T-LIGHT HOLDER     2028
REGENCY CAKESTAND 3 TIER               1724
JUMBO BAG RED RETROSPOT                1618
ASSORTED COLOUR BIRD ORNAMENT          1408
PARTY BUNTING                          1397
                                       ... 
EASTER CRAFT IVY WREATH WITH CHICK        1
OCEAN STRIPE HAMMOCK                      1
INCENSE BAZAAR PEACH                      1
BLACK VINT ART DEC CRYSTAL BRACELET       1
BLUE PADDED SOFT MOBILE                   1
Name: count, Length: 3877, dtype: int64


In [ ]:
# En çok sipariş edilen 5 ürün
top_5_products = df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity",ascending=False).head()
print(top_5_products)

                                    Quantity
Description                                 
PAPER CRAFT , LITTLE BIRDIE            80995
MEDIUM CERAMIC TOP STORAGE JAR         77916
WORLD WAR 2 GLIDERS ASSTD DESIGNS      54415
JUMBO BAG RED RETROSPOT                46181
WHITE HANGING HEART T-LIGHT HOLDER     36725


In [ ]:
# Faturadaki her bir ürüne toplam ne kadar bedeş ödendiğini belirleyelim
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [ ]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom,15.3000
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom,22.0000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
...,...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.1000,12680.0000,France,12.6000
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.1500,12680.0000,France,16.6000
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.1500,12680.0000,France,16.6000
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.9500,12680.0000,France,14.8500


In [ ]:
#####################################################################
# RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
#####################################################################

In [ ]:
# Recency, Frequency, Monetary

In [ ]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [ ]:
today_date = dt.datetime(2011, 12, 11)   # Analizi yaptığımız gün diye varsayalım

In [ ]:
type(today_date)

datetime.datetime

In [ ]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom,15.3000
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom,22.0000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400


In [ ]:
rfm =df.groupby("Customer ID").agg({"InvoiceDate": lambda InvoiceDate: (today_date - InvoiceDate.max()).days,   # Son alışverişten bugüne kaç gün geçti
                                    "Invoice": lambda Invoice: Invoice.nunique(),       # Müşteri kaç kez alışveriş yapmış
                                    "TotalPrice": lambda TotalPrice: TotalPrice.sum()})     # Müşteri ne kadar harcama yapmış

In [ ]:
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.0000,326,1,77183.6000
12347.0000,3,7,4310.0000
12348.0000,76,4,1797.2400
12349.0000,19,1,1757.5500
12350.0000,311,1,334.4000


In [ ]:
rfm.columns = ["Recency", "Frequency", "Monetary"]

In [ ]:
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.0000,326,1,77183.6000
12347.0000,3,7,4310.0000
12348.0000,76,4,1797.2400
12349.0000,19,1,1757.5500
12350.0000,311,1,334.4000


In [ ]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4339.0000,93.0415,100.0078,1.0000,18.0000,51.0000,142.5000,374.0000
Frequency,4339.0000,4.2720,7.7055,1.0000,1.0000,2.0000,5.0000,210.0000
Monetary,4339.0000,2053.7972,8988.2478,0.0000,307.2450,674.4500,1661.6400,280206.0200


In [ ]:
rfm = rfm[rfm["Monetary"] > 0]

In [ ]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4338.0000,93.0595,100.0123,1.0000,18.0000,51.0000,142.7500,374.0000
Frequency,4338.0000,4.2727,7.7062,1.0000,1.0000,2.0000,5.0000,210.0000
Monetary,4338.0000,2054.2706,8989.2299,3.7500,307.4150,674.4850,1661.7400,280206.0200


In [ ]:
#################################################################
# RFM Skorlarının Hesaplanması (Calculating RFM Scores)
#################################################################

In [ ]:
rfm["Recency_Score"] = pd.qcut(rfm["Recency"], 5, labels=[5, 4, 3, 2, 1])

In [ ]:
rfm["Frequency_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
# rank (method="first") diyerek ilk gördüğünü ilk sınıfa ata işlemi yapılıyor

In [ ]:
rfm["Monetary_Score"] = pd.qcut(rfm["Monetary"], 5, labels=[1, 2, 3, 4, 5])

In [ ]:
rfm.head()

,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score
Customer ID,,,,,,
12346.0000,326,1,77183.6000,1,1,5
12347.0000,3,7,4310.0000,5,5,5
12348.0000,76,4,1797.2400,2,4,4
12349.0000,19,1,1757.5500,4,1,4
12350.0000,311,1,334.4000,1,1,2


In [ ]:
# RF_Scoru 'nun oluşturulması
rfm["RFM_SCORE"] = (rfm["Recency_Score"].astype(str) +
                    rfm["Frequency_Score"].astype(str))

In [ ]:
rfm.head()

,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RFM_SCORE
Customer ID,,,,,,,
12346.0000,326,1,77183.6000,1,1,5,11
12347.0000,3,7,4310.0000,5,5,5,55
12348.0000,76,4,1797.2400,2,4,4,24
12349.0000,19,1,1757.5500,4,1,4,41
12350.0000,311,1,334.4000,1,1,2,11


In [ ]:
###############################################################################################
# RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
###############################################################################################

In [ ]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [ ]:
rfm["Segment"] = rfm["RFM_SCORE"].replace(seg_map, regex=True)
rfm.head(10)

,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.0000,326,1,77183.6000,1,1,5,11,hibernating
12347.0000,3,7,4310.0000,5,5,5,55,champions
12348.0000,76,4,1797.2400,2,4,4,24,at_Risk
12349.0000,19,1,1757.5500,4,1,4,41,promising
12350.0000,311,1,334.4000,1,1,2,11,hibernating
12352.0000,37,8,2506.0400,3,5,5,35,loyal_customers
12353.0000,205,1,89.0000,1,1,1,11,hibernating
12354.0000,233,1,1079.4000,1,1,4,11,hibernating
12355.0000,215,1,459.4000,1,1,2,11,hibernating


In [ ]:
# Case 1: Önemli gördüğünüz 3 segmenti seçiniz. Bu üç segmenti hem aksiyon kararları açısından hemde segmentlerin yapısı açısından(ortalama RFM değerleri) yorumlayınız.

In [ ]:
# champions : En yakın zamanda alışveriş yapmış, en sık alışveriş yapan müşteri grubu
rfm[rfm["Segment"] == "champions"].agg({"Recency" : "mean",
                                        "Frequency" : "mean",
                                        "Monetary" : "mean"})

,0
Recency,6.3618
Frequency,12.4171
Monetary,6857.9639


In [ ]:
# Bu müşterilere özel indirimler veya VIP üyelikler sunabiliriz.
# Bu müşteri grubu aynı zamanda bizi başkalarına önermeye daha açıktır.
# Upsell ve cross-sell kampanyaları yapılabilir.

In [ ]:
# at_Risk : Geçmişte alışveriş yapmış ama uzun süredir sessiz, harcama potansiyeli olan müşteri grubu
rfm[rfm["Segment"] == "at_Risk"].agg({"Recency": "mean",
                                      "Frequency": "mean",
                                      "Monetary": "mean"})

,0
Recency,153.7858
Frequency,2.8786
Monetary,1084.5353


In [ ]:
# Bu müşterilere özel "Sizi özledik!" tarzında e-postalar gönderilebilir.
# Sınırlı süreli özel indirim teklifleri
# Sınırlı sayıda düzenli indirimlerle geri dönüş teşviki yapılabilir.

In [ ]:
# new_customers : İlk alışverişni yakın zamanda yapmış, tek seferlik işlemle müşteri olmuş bu yüzden davranış hareketi bilinmiyor

In [ ]:
rfm[rfm["Segment"] == "new_customers"].agg({"Recency": "mean",
                                            "Frequency": "mean",
                                            "Monetary": "mean"})

,0
Recency,7.4286
Frequency,1.0000
Monetary,388.2129


In [ ]:
# Hoş geldin e-postaları ve rehber içerikler sunulabilir.
# Tekrar satın alım için aışverişlerde %20 gibi indirimler yapılabilir.
# Kullanıcı deneyimini iyileştirme ve geri bildirim isteme yapılabilir.

In [ ]:
# Case 2: "Loyal Customers" sınıfına ait customer ID'leri seçerek excel çıktısını alınız.

In [ ]:
loyal_customers = rfm[rfm["Segment"] == "loyal_customers"]
loyal_customers_ids = loyal_customers.reset_index()[["Customer ID"]]
loyal_customers_ids.to_excel("loyal_customers.xlsx", index=False)